## Installation de bibliotheques 

**pip install beautifulsoup4**

**pip install requests**

**pip install PyPDF2**

**pip install pdfplumber**


In [50]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
from datetime import datetime

import pdfplumber
import re, io
import PyPDF2
from io import BytesIO
import os



## 1. Scraper les articles bioRxiv liés à la bioinformatique 

Parcourir les articles de la collection Bioinformatics contenant au total 2153 pages d'articles, ainsi scraper les 10 pages à chaque exécution en enregistrant la dernière page scrappée et leurs informations dans la database.

In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
from datetime import datetime

url = 'https://www.biorxiv.org'

# créer la base de données SQLite "bioinformatics_articles" 
database = sqlite3.connect('bioinformatics_articles.db')
c = database.cursor() 

# créer la table 'articles' si elle n'existe pas 
c.execute('''CREATE TABLE IF NOT EXISTS articles
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
             title TEXT, 
             link TEXT,
             doi TEXT UNIQUE,
             date TEXT,
             pdf_link TEXT,
             abstract TEXT)''')

# Créer une table pour stocker la dernière page scrappée
c.execute('''CREATE TABLE IF NOT EXISTS scrap_info (
             last_page INTEGER)''')

# Récupérer la dernière page scrappée
c.execute("SELECT last_page FROM scrap_info")
result = c.fetchone()
last_page = result[0] if result else 0  # Si aucune page n'a été scrappée, commencer à la page 1
start_page = last_page + 1

## Nombre de pages à scraper (exemple : 10 pages à la fois)
pages_to_scrap_per_run = 10
total_pages = min(start_page + pages_to_scrap_per_run - 1, 2153)  # Limiter à 2153 pages

# boucle qui parcourt chaque page de la collection Bioinformatics
for page in range(start_page, total_pages + 1):
    print(f"Scraping page {page} sur {total_pages}")

    # requête qui récupère la page bioinfo actuelle
    response = requests.get(f'{url}/collection/bioinformatics?page={page}')
    soup = BeautifulSoup(response.content, 'html.parser')

    # Trouver les liens vers les articles sur cette page
    articles = soup.find_all('a', class_='highwire-cite-linked-title')
    print(f'Nombre total d\'articles trouvés sur la page {page} : {len(articles)}')

    # Parcourir tous les articles de la page et enregistrer leurs titres et liens
    for article in articles:
        title = article.text.strip()
        link = url + article['href']

        # Requête pour accéder à la page de l'article
        article_response = requests.get(link)
        article_soup = BeautifulSoup(article_response.content, 'html.parser')

        # extraire le DOI
        doi_tag = article_soup.find('meta', {'name': 'citation_doi'})
        doi = doi_tag['content'] if doi_tag else 'DOI non disponible'

        # extraire la date de publication
        date_tag = article_soup.find('meta', {'name': 'citation_publication_date'})
        date = date_tag['content'] if date_tag else 'Date non disponible'
        if date != 'Date non disponible':
            date = date.replace('/', '-')
            date = datetime.strptime(date, '%Y-%m-%d').date().isoformat()

        # extraire le lien vers le fichier PDF
        pdf_links = article_soup.find_all('a')
        pdf_link = next((url + pdf['href'] for pdf in pdf_links if 'PDF' in pdf.text and pdf['href'].endswith('.pdf')), 'Lien PDF non disponible')

        # Extraire l'abstract
        abstract_tag = article_soup.find('meta', {'name': 'citation_abstract'})
        clean_abstract = abstract_tag['content'] if abstract_tag else 'Abstract non disponible'

        # Vérifier si l'article existe déjà dans la base de données
        c.execute("SELECT * FROM articles WHERE doi = ? AND date = ?", (doi, date))
        result = c.fetchone()

        if result:
            print(f"L'article avec DOI {doi} existe déjà, ignoré.")
        else:
            # Insérer l'article dans la base de données
            c.execute("INSERT INTO articles (title, link, doi, date, pdf_link, abstract) VALUES (?, ?, ?, ?, ?, ?)", 
                      (title, link, doi, date, pdf_link, clean_abstract))
            print(f"Article ajouté : {title}")

        time.sleep(1)  # pause d'une seconde pour éviter de surcharger le serveur

    # Sauvegarder les modifications dans la base de données après chaque page
    database.commit()

    # Mettre à jour la dernière page scrappée dans la table 'scrap_info'
    c.execute("UPDATE scrap_info SET last_page = ?", (page,))
    if c.rowcount == 0:  
        c.execute("INSERT INTO scrap_info (last_page) VALUES (?)", (page,))

# Sauvegarder la dernière page dans la base de données
database.commit()

# Fermer la connexion à la base de données
database.close()

## 2. Extraire les URLs des dépôts logiciels(github and gitlab) dans les abstracts ET les fichiers PDF si url valide

In [ ]:

# Connexion à la base de données SQLite
database = sqlite3.connect('bioinformatics_articles.db')
c = database.cursor()

# Vérifier et ajouter les colonnes software_links et processed si elles n'existent pas
try:
    c.execute("ALTER TABLE articles ADD COLUMN software_links TEXT")
except sqlite3.OperationalError:
    pass  # La colonne existe déjà

try:
    c.execute("ALTER TABLE articles ADD COLUMN processed INTEGER DEFAULT 0")
except sqlite3.OperationalError:
    pass  # La colonne existe déjà

# Fonction pour vérifier si un lien est valide et s'il est un lien GitHub ou GitLab
def is_valid_link(link):
    return link.startswith('http://') or link.startswith('https://')

def is_github_or_gitlab(link):
    return 'github.com' in link or 'gitlab.com' in link

# Fonction pour nettoyer les liens partiels ou incorrects
def clean_link(link):
    link = link.strip(').;,[]')  # Supprimer les caractères indésirables
    if is_valid_link(link) and len(link) > len('https://github.com/'):
        return link
    return None

# Fonction pour tester la validité des URLs via une requête HTTP
def check_url_validity(link):
    try:
        response = requests.get(link, timeout=10)
        if response.status_code == 200:
            print(f"Le lien est valide : {link}")
            return True
        else:
            print(f"Le lien a retourné un code de statut {response.status_code} pour : {link}")
            return False
    except requests.RequestException:
        print(f"Erreur lors de la vérification du lien : {link}")
        return False

# Fonction pour extraire les URLs des dépôts logiciels
def extract_repository_urls(text):
    url_pattern = r'(https?://[^\s]+)'
    urls = re.findall(url_pattern, text)
    repo_urls = []

    for url in urls:
        cleaned_url = clean_link(url)
        if cleaned_url and is_github_or_gitlab(cleaned_url):
            if check_url_validity(cleaned_url):  # Vérification de l'URL avant de l'ajouter
                repo_urls.append(cleaned_url)

    return repo_urls

# Fonction pour extraire les liens de dépôt logiciel
def extract_software_links(article_id, pdf_link, abstract):
    software_links_set = set()

    if pdf_link and pdf_link != 'Lien PDF non disponible':
        try:
            print(f"Ouverture du PDF pour l'article {article_id}: {pdf_link}")
            response = requests.get(pdf_link)
            response.raise_for_status()

            with pdfplumber.open(io.BytesIO(response.content)) as pdf:
                print(f"Le PDF a été ouvert avec succès pour l'article {article_id}. Recherche de liens...")
                for page in pdf.pages:
                    text = page.extract_text()
                    if text:
                        links_from_pdf = extract_repository_urls(text)
                        if links_from_pdf:
                            software_links_set.update(links_from_pdf)
                            print(f"Liens trouvés dans le PDF pour l'article {article_id} : {links_from_pdf}")

        except Exception as e:
            print(f"Erreur lors de l'ouverture du PDF pour l'article {article_id}: {e}")

    if abstract and abstract != 'Abstract non disponible':
        print(f"Vérification de l'abstract pour l'article {article_id}.")
        links_from_abstract = extract_repository_urls(abstract)
        if links_from_abstract:
            software_links_set.update(links_from_abstract)
            print(f"Liens trouvés dans l'abstract pour l'article {article_id} : {links_from_abstract}")

    valid_links = [clean_link(link) for link in software_links_set if clean_link(link) and is_github_or_gitlab(link)]

    if valid_links:
        print(f"Liens de dépôt logiciel valides pour l'article {article_id} : {valid_links}")
        c.execute("UPDATE articles SET software_links = ?, processed = 1 WHERE id = ?", (', '.join(valid_links), article_id))
        print(f"Liens ajoutés à la base de données pour l'article {article_id}.")
    else:
        print(f"Aucun lien de dépôt logiciel trouvé pour l'article {article_id}.")

    time.sleep(1)

# Fonction pour traiter uniquement les nouveaux articles
def process_new_articles():
    c.execute("SELECT id, title, pdf_link, abstract FROM articles WHERE processed = 0")
    articles_to_process = c.fetchall()

    for article in articles_to_process:
        article_id, title, pdf_link, abstract = article
        print(f"Traitement de l'article : {title} (ID: {article_id})...")
        extract_software_links(article_id, pdf_link, abstract)

# Appel de la fonction principale
try:
    process_new_articles()
except KeyboardInterrupt:
    print("Traitement interrompu. Les données sont sauvegardées.")
finally:
    database.close()

##  Affichage de mes articles dans la base de données (A supprimer c'est juste pr que je vois bien l'ajout sur jupyter)

In [ ]:
#le faire en cas de soucis and need tout recommencer 
#c.execute("DROP TABLE IF EXISTS articles")

In [ ]:
import sqlite3

# Connexion à la base de données SQLite
database = sqlite3.connect('bioinformatics_articles.db')
c = database.cursor()

# Fonction pour afficher les attributs de la table articles
def display_articles():
    # Exécuter une requête pour récupérer toutes les colonnes et enregistrements de la table articles
    c.execute("SELECT * FROM articles")
    
    # Récupérer les noms de colonnes
    column_names = [description[0] for description in c.description]
    
    # Afficher les noms des colonnes
    print(f"{' | '.join(column_names)}")
    print("-" * (len(column_names) * 4))  # Ligne de séparation
    
    # Récupérer et afficher tous les enregistrements
    for row in c.fetchall():
        print(' | '.join(map(str, row)))

# Appeler la fonction pour afficher les articles
display_articles()

# Fermer la connexion à la base de données
database.close()

# suite du projet  :

2.	Archivage via Software Heritage:

Utiliser l’API de Software Heritage pour vérifier si le dépôt logiciel est déjà archivé dans leur base de données. Si le dépôt n’est pas encore archivé, soumettre automatiquement une demande d’archivage via l’API.

--->Script 3 : lire la BDD, récupérer les URL, vérifier que le dépôt est archivé dans Software Heritage
